In [2]:
import numpy as np
import tensorflow as tf
import torch as th
from spektral.layers.convolutional import ARMAConv as ARMAConvSpektral
from torch_geometric.nn import ARMAConv as ARMAConvTorch

/Users/vitor/Desktop/mestrado/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [13]:
a_input = np.load('a_input.npy')

In [14]:
a_input.shape

(512, 3, 3)

In [34]:
b = a_input[:2]
b

array([[[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]],

       [[0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 0.]]])

In [33]:
b.reshape(2*3, 3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 0.]])

In [30]:
node_features = np.load('temporal_train.npy')

In [12]:
node_features.shape

(512, 3, 48)

In [25]:
node_features[1]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [191]:
adjacency_graph = tf.convert_to_tensor(
    [
        [
            [1., 0., 0.],
            [0., 1., 0.],
            [0., 0., 1.]
        ],
        [
            [0., 2., 3., 5.],
            [2., 0., 4., 0.],
            [3., 4., 0., 0.],
            [5., 4., 0., 0.],
        ],
    ]
)

node_features = tf.convert_to_tensor(
    [
        [1.0, 2.0, 3.0],
        [1.0, 2.0, 3.0],
        [1.0, 2.0, 3.0],
    ]
)

adjacency_graph = tf.convert_to_tensor(

    [
        [1., 3., 0.],
        [3., 4., 2.],
        [0., 2., 0.],
    ],

)

print(node_features.shape)
print(adjacency_graph.shape)

arma = ARMAConvSpektral(node_features.shape[-1], shared_weights=True)
arma([node_features, adjacency_graph])

SyntaxError: invalid syntax (293872022.py, line 18)

In [185]:
import torch
from torch_geometric.nn import ARMAConv
from torch_geometric.data import Data

# Sample data preparation
num_nodes = 10
num_node_features = 5
num_classes = 3
edges = torch.tensor([[0, 1, 1, 2],
                      [1, 2, 0, 3]], dtype=torch.long)
node_features = torch.tensor([
    [1.0, 2.0, 3.0, 4.0, 10.0],
    [1.0, 2.0, 3.0, 4.0, 10.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
    [1.0, 2.0, 3.0, 4.0, 5.0],
])

# Graph data object
data = Data(x=node_features, edge_index=edges)


# Model
class PyGARMA(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.arma = ARMAConv(num_node_features, num_classes, num_stacks=1, num_layers=1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.arma(x, edge_index)
        return x


model_pyg = PyGARMA()
model_pyg.train()
out = model_pyg(data)
print(out)

tensor([[0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.1281],
        [0.0000, 0.0000, 1.1281],
        [0.0000, 0.0000, 1.1281],
        [0.0000, 0.0000, 1.1281],
        [0.0000, 0.0000, 1.1281],
        [0.0000, 0.0000, 1.1281]], grad_fn=<MeanBackward1>)


In [181]:
import numpy as np
from spektral.layers import ARMAConv
from tensorflow.keras import Model
from tensorflow.keras.layers import Input
from spektral.data import Graph

# Converting PyTorch data to Spektral graph
x = np.array(node_features)
a = np.zeros((num_nodes, num_nodes))
a[edges[0], edges[1]] = 1

graph = Graph(x=x, a=a)


# Spektral model
class SpektralARMA(Model):
    def __init__(self):
        super().__init__()
        self.arma = ARMAConv(channels=num_classes, n_stacks=1, n_layers=1)

    def call(self, inputs):
        x, a = inputs
        return self.arma([x, a])


x_in = Input(shape=(num_node_features,))
a_in = Input((num_nodes,))
model_spektral = SpektralARMA()
model_spektral([x_in, a_in])
model_spektral.compile()
out = model_spektral([graph.x, graph.a])
print(out)


tf.Tensor(
[[  0.          0.        143.30136  ]
 [  0.          0.        147.9425   ]
 [  0.          2.176948    9.282283 ]
 [  0.          1.088474    4.6411414]
 [  0.          1.088474    4.6411414]
 [  0.          1.088474    4.6411414]
 [  0.          1.088474    4.6411414]
 [  0.          1.088474    4.6411414]
 [  0.          1.088474    4.6411414]
 [  0.          1.088474    4.6411414]], shape=(10, 3), dtype=float32)


/Users/vitor/Desktop/mestrado/venv/lib/python3.9/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [6]:
import torch

# Example adjacency matrices as dense tensors
adj_matrices = torch.tensor([
    [[1., 0., 0.],
     [0., 1., 1.],
     [0., 1., 1.]],

    [[0., 1., 0.],
     [1., 0., 1.],
     [0., 1., 0.]],
    
    [[0., 1., 0.],
     [1., 0., 1.],
     [0., 1., 0.]]
])

# Convert adjacency matrices to edge_index format and edge_type
num_nodes = adj_matrices.size(1)
edge_index = []
edge_type = []

for i, adj in enumerate(adj_matrices):
    row, col = torch.where(adj > 0)
    edge_index.append(torch.stack([row, col], dim=0))
    edge_type.append(torch.full((row.size(0),), i, dtype=torch.long))

# Concatenate all edges
edge_index = torch.cat(edge_index, dim=1)
edge_type = torch.cat(edge_type)

print(edge_index)
print(edge_type)


tensor([[0, 1, 1, 2, 2, 0, 1, 1, 2, 0, 1, 1, 2],
        [0, 1, 2, 1, 2, 1, 0, 2, 1, 1, 0, 2, 1]])
tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2])


In [218]:
def adjacency_to_edge_index(adj_matrix):
    """
    Converts an adjacency matrix to an edge_index tensor suitable for use with PyTorch Geometric,
    considering only one entry for each undirected edge.

    Parameters:
    adj_matrix (np.ndarray or torch.Tensor): The adjacency matrix of shape (num_nodes, num_nodes)

    Returns:
    torch.Tensor: The edge_index tensor of shape (2, num_edges)
    """
    # Ensure the adjacency matrix is a numpy array
    if isinstance(adj_matrix, torch.Tensor):
        adj_matrix = adj_matrix.numpy()
    
    # Find indices of non-zero elements in the upper triangle of the matrix
    num_nodes = adj_matrix.shape[0]
    src_nodes, dst_nodes = np.triu_indices(num_nodes, k=0)  # k=1 starts from the first off-diagonal
    weights = adj_matrix[src_nodes, dst_nodes]  # Extract weights
    mask = weights > 0  # Mask for non-zero entries in the upper triangle

    # Filter the source and destination nodes based on the mask
    src_nodes = src_nodes[mask]
    dst_nodes = dst_nodes[mask]
    weights = weights[mask]  # Filter weights based on the mask

    # Convert the indices to a PyTorch tensor
    edge_index = torch.tensor([src_nodes, dst_nodes], dtype=torch.long)
    edge_weight = torch.tensor(weights, dtype=torch.float)  # Convert weights to a tensor
    
    return edge_index, edge_weight

def process_batch_of_matrices(batch_adj_matrices):
    """
    Processes a batch of adjacency matrices, converting each to edge_index and edge_weight tensors.

    Parameters:
    batch_adj_matrices (list of np.ndarray or list of torch.Tensor): List of adjacency matrices

    Returns:
    combined_edge_index (torch.Tensor): Combined edge_index tensor for all graphs in the batch
    combined_edge_weights (torch.Tensor): Combined edge weights tensor for all graphs in the batch
    """
    batch_edge_indices = []
    batch_edge_weights = []
    node_offset = 0

    for adj_matrix in batch_adj_matrices:
        edge_index, edge_weights = adjacency_to_edge_index(adj_matrix)
        # Adjust edge indices by the current node offset
        edge_index[0, :] += node_offset
        edge_index[1, :] += node_offset
        batch_edge_indices.append(edge_index)
        batch_edge_weights.append(edge_weights)
        # Update the node offset
        node_offset += adj_matrix.shape[0]  # Increment the node offset by the number of nodes in the current graph

    # Combine all edge indices and weights from the batch into single tensors
    combined_edge_index = torch.cat(batch_edge_indices, dim=1)
    combined_edge_weights = torch.cat(batch_edge_weights, dim=0)
    
    return combined_edge_index, combined_edge_weights


In [7]:
def adjacency_to_edge_index_with_weights(adj_matrix):
    if isinstance(adj_matrix, torch.Tensor):
        adj_matrix = adj_matrix.numpy()
    num_nodes = adj_matrix.shape[0]
    src_nodes, dst_nodes = np.triu_indices(num_nodes, k=0)
    weights = adj_matrix[src_nodes, dst_nodes]
    mask = weights > 0
    src_nodes = src_nodes[mask]
    dst_nodes = dst_nodes[mask]
    weights = weights[mask]
    return torch.tensor([src_nodes, dst_nodes], dtype=torch.long), torch.tensor(weights, dtype=torch.float)

def process_batch_of_matrices(batch_adj_matrices):
    """
    Processes a batch of adjacency matrices, each converted to separate edge_index and edge_weight tensors.

    Parameters:
    batch_adj_matrices (list of np.ndarray or list of torch.Tensor): List of adjacency matrices

    Returns:
    list of tuples: Each tuple contains (edge_index, edge_weight) for each graph in the batch
    """
    results = []

    for adj_matrix in batch_adj_matrices:
        edge_index, edge_weights = adjacency_to_edge_index_with_weights(adj_matrix)
        results.append((edge_index, edge_weights))

    return results

def pad_matrices(batch_results, max_edges=3):
    """
    Pad the edge indices and weights of graphs to ensure all have the same number of edges (max_edges).

    Parameters:
    batch_results (list of tuples): Each tuple contains (edge_index, edge_weight) for each graph
    max_edges (int): Maximum number of edges per graph matrix

    Returns:
    np.ndarray: Array of shape (num_graphs, 2, max_edges) for edge indices
    np.ndarray: Array of shape (num_graphs, max_edges) for edge weights
    """
    num_graphs = len(batch_results)
    padded_edge_indices = np.zeros((num_graphs, 2, max_edges), dtype=int)
    padded_edge_weights = np.zeros((num_graphs, max_edges), dtype=float)

    for i, (edge_index, edge_weights) in enumerate(batch_results):
        num_edges = edge_index.shape[1]
        if num_edges > 0:
            num_to_copy = min(num_edges, max_edges)
            padded_edge_indices[i, :, :num_to_copy] = edge_index[:, :num_to_copy]
            padded_edge_weights[i, :num_to_copy] = edge_weights[:num_to_copy]
    
    return padded_edge_indices, padded_edge_weights

(array([[[0, 0, 0],
         [1, 0, 0]],
 
        [[0, 1, 0],
         [1, 2, 0]],
 
        [[0, 1, 0],
         [1, 2, 0]],
 
        [[0, 1, 0],
         [1, 2, 0]],
 
        [[1, 2, 0],
         [1, 2, 0]],
 
        [[0, 0, 1],
         [1, 2, 1]]]),
 array([[1.        , 0.        , 0.        ],
        [0.70710677, 0.70710677, 0.        ],
        [0.70710677, 0.70710677, 0.        ],
        [0.70710677, 0.70710677, 0.        ],
        [1.        , 1.        , 0.        ],
        [0.40824831, 0.57735026, 0.75      ]]))

In [234]:
a_input[2:8]

array([[[0.        , 1.        , 0.        ],
        [1.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ]],

       [[0.        , 0.70710678, 0.        ],
        [0.70710678, 0.        , 0.70710678],
        [0.        , 0.70710678, 0.        ]],

       [[0.        , 0.70710678, 0.        ],
        [0.70710678, 0.        , 0.70710678],
        [0.        , 0.70710678, 0.        ]],

       [[0.        , 0.70710678, 0.        ],
        [0.70710678, 0.        , 0.70710678],
        [0.        , 0.70710678, 0.        ]],

       [[0.        , 0.        , 0.        ],
        [0.        , 1.        , 0.        ],
        [0.        , 0.        , 1.        ]],

       [[0.        , 0.40824829, 0.57735027],
        [0.40824829, 0.75      , 0.        ],
        [0.57735027, 0.        , 0.        ]]])

In [30]:
def adjacency_to_edge_index_with_weights(adj_matrix):
    if isinstance(adj_matrix, torch.Tensor):
        adj_matrix = adj_matrix.numpy()
    num_nodes = adj_matrix.shape[0]
    src_nodes, dst_nodes = np.triu_indices(num_nodes, k=0)
    weights = adj_matrix[src_nodes, dst_nodes]
    mask = weights > 0
    src_nodes = src_nodes[mask]
    dst_nodes = dst_nodes[mask]
    weights = weights[mask]
    return torch.tensor([src_nodes, dst_nodes], dtype=torch.long), torch.tensor(weights, dtype=torch.float)

def prepare_pyg_batch(batch_adj_matrices, max_edges=3):
    """
    Adjusts edge indices and concatenates them for a batch of graphs, preparing for PyG DataLoader.
    
    Parameters:
    batch_adj_matrices (list of np.ndarray): List of adjacency matrices.
    
    Returns:
    edge_indices (torch.Tensor): All edge indices combined, adjusted for cumulative node counts.
    edge_weights (torch.Tensor): All edge weights combined.
    """
    num_graphs = len(batch_adj_matrices)
    cumulative_nodes = 0
    all_edge_indices = []
    all_edge_weights = []

    for adj_matrix in batch_adj_matrices:
        edge_index, edge_weights = adjacency_to_edge_index_with_weights(adj_matrix)
        # Adjust edge indices based on the cumulative number of nodes processed
        edge_index[0, :] += cumulative_nodes
        edge_index[1, :] += cumulative_nodes
        all_edge_indices.append(edge_index)
        all_edge_weights.append(edge_weights)
        # Update the cumulative nodes count
        cumulative_nodes += adj_matrix.shape[0]

    # Concatenate all edge indices and weights into single tensors
    combined_edge_index = torch.cat(all_edge_indices, dim=1)
    combined_edge_weights = torch.cat(all_edge_weights, dim=0)
    
    

    return combined_edge_index, combined_edge_weights

In [35]:
prepare_pyg_batch(a_input[:4])

(tensor([[ 0,  1,  2,  3,  6,  9, 10],
         [ 0,  1,  2,  4,  7, 10, 11]]),
 tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.7071, 0.7071]))

7

In [18]:
#parse tensor vectors to arrays 
print(np.array(cc))

[[ 0  1  2  3  6  9 10]
 [ 0  1  2  4  7 10 11]]


In [22]:
a_input[:1]

array([[[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]])

In [29]:
torch.tensor([1,2,3]).size(0)

3